# Illustrations and comparisons of MPP and DIV encodings

In [ ]:
from geo_encodings import MPPEncoder
from geo_encodings import DIVEncoder
from geo_encodings import draw_shape

import shapely
from shapely.ops import nearest_points

import plotly
import plotly.subplots
from plotly.graph_objs import Scatter

## Define a geometry

In [ ]:
# Pick a shape to work with. Uncomment a line below.

# g = shapely.wkt.loads('POINT(287 122)')
g = shapely.wkt.loads('LINESTRING(40 70, 160 110, 213 190, 285 205)')
# g = shapely.wkt.loads('POLYGON((70 20, 210 105, 127 240, 70 20))')

# g = shapely.wkt.loads('MULTIPOINT(287 122, 113 207)')
# g = shapely.wkt.loads('MULTILINESTRING((40 70, 160 110, 213 190, 285 205), (220 140, 305 110, 345 30))')
# g = shapely.wkt.loads('MULTIPOLYGON(((70 20, 210 105, 127 240, 70 20)), ((260 225, 270 170, 330 95, 335 185, 260 225)))')


## DIV encoding

In [ ]:
x0, y0 = 0, 0
x1, y1 = 300, 300
resolution = 50
encoder = DIVEncoder([x0, y0, x1, y1], resolution)
print('%d tiles in encoder' % len(encoder))

encoding = encoder.encode(g).values()
print('DIV encoding = [')
for i in range(3):
    s = '   '
    for j in range(12):
        k = i * 12 + j
        s = '%s %d' % (s, encoding[k])
    print(s)
print(']\n')
        

In [ ]:

fig = plotly.subplots.make_subplots(1, 1)

found0 = False
found1 = False

for tile, e in list(zip(encoder.tiles, encoding)):
    coords = tile.exterior.coords

    # All this is for getting plotly to only show the legend entry once per
    # tile type. 
    show = False
    if e:
        name = 'intersection = 1'
        color = 'blue'
        if found1 == False:
            show = True
            found1 = True
    else:
        name = 'intersection = 0'
        color = 'gray'
        if found0 == False:
            show = True
            found0 = True

    # Draw all the tiles, colored according to whether they intersect the shape.
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    trace = Scatter(
        x=xx, y=yy, name=name,
        mode='lines', marker={'color': color}, fill='toself',
        showlegend=show
    )
    fig.append_trace(trace, 1, 1)

# Draw the shape.
coords = g.coords
xx = [z[0] for z in coords]
yy = [z[1] for z in coords]
trace = Scatter(
    x=xx, y=yy, name=g.geom_type, 
    mode='lines', marker={'color': 'black'}, line={'width': 5}
)
fig.append_trace(trace, 1, 1)

fig['layout']['width'] = 600
fig['layout']['height'] = 500
fig['layout']['xaxis1']['title'] = 'x coordinate'    
fig['layout']['yaxis1']['title'] = 'y coordinate'    
fig

## MPP encoding

In [ ]:
x0, y0 = 0, 0
x1, y1 = 300, 300
resolution = 50
encoder = MPPEncoder([x0, y0, x1, y1], resolution, center=True)
print('%d reference points in encoder' % len(encoder))

encoding = encoder.encode(g).values()
print('MPP encoding = [')
for i in range(3):
    s = '   '
    for j in range(12):
        k = i * 12 + j
        s = '%s %.2f' % (s, encoding[k])
    print(s)
print(']\n')


In [ ]:
geom = g
gt = geom.geom_type

fig = plotly.subplots.make_subplots(1, 1)

xx = []
yy = []
for rp in encoder.ref_points:
    p = nearest_points(geom, rp)[0]
    connector_x = [p.xy[0][0], rp.xy[0][0]]
    connector_y = [p.xy[1][0], rp.xy[1][0]]
    xx = xx + connector_x + [None]
    yy = yy + connector_y + [None]
trace = Scatter(
    x=xx, y=yy, name='distances',
    mode='lines', line={'dash': 'dash', 'color': 'gray'}
)
fig.append_trace(trace, 1, 1)

# Draw the reference points.
trace = Scatter(
    x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
    mode='markers', marker={'color': 'blue', 'size': 10}
)
fig.append_trace(trace, 1, 1)

if gt == 'Polygon':
    coords = geom.exterior.coords
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    trace = Scatter(
        x=xx, y=yy, name=gt, 
        mode='lines', marker={'color': 'red'}, fill='toself'
    )
    fig.append_trace(trace, 1, 1)

elif gt == 'MultiPolygon':
    for g in geom.geoms:
        coords = g.exterior.coords
        xx = [z[0] for z in coords]
        yy = [z[1] for z in coords]
        trace = Scatter(
            x=xx, y=yy, name=gt, 
            mode='lines', marker={'color': 'red'}, fill='toself'
        )
        fig.append_trace(trace, 1, 1)

elif gt == 'Point':
    xx = [geom.xy[0][0]]
    yy = [geom.xy[1][0]]
    trace = Scatter(
        x=xx, y=yy, name=gt, 
        mode='markers', marker={'color': 'red', 'size': 12}
    )
    fig.append_trace(trace, 1, 1)
elif gt == 'MultiPoint':
    xx = [z.xy[0][0] for z in geom.geoms]
    yy = [z.xy[1][0] for z in geom.geoms]
    trace = Scatter(
        x=xx, y=yy, name=gt, 
        mode='markers', marker={'color': 'red', 'size': 12}
    )
    fig.append_trace(trace, 1, 1)
elif gt == 'LineString':
    coords = geom.coords
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    # zp.xy(xx, yy, mode='ml', color='red', name=gt, fill='none')
    trace = Scatter(
        x=xx, y=yy, name=gt, 
        mode='lines', marker={'color': 'black'}, line={'width': 5}
    )
    fig.append_trace(trace, 1, 1)
elif gt == 'MultiLineString':
    xx = []
    yy = []
    for g in geom.geoms:
        coords = g.coords
        xx += [z[0] for z in coords] + [None]
        yy += [z[1] for z in coords] + [None]
    trace = Scatter(
        x=xx, y=yy, name=gt, 
        mode='lines', marker={'color': 'red', 'size': 12}
    )
    fig.append_trace(trace, 1, 1)

fig['layout']['width'] = 600
fig['layout']['height'] = 500
fig['layout']['xaxis1']['title'] = 'x coordinate'    
fig['layout']['yaxis1']['title'] = 'y coordinate'    
fig['layout']['xaxis1']['range'] = [0, 300]    
fig['layout']['yaxis1']['range'] = [0, 300]    
fig